# Mining sites with polygons and ownership information

- An open database on global coal  and metal mine production (Jasanski et al., 2023)
- Global-scale mining polygons (Version 2) (Maus et al. 2022)

## Set up

In [ ]:
# In case users want to run the Notebook using their GDrive
from google.colab import drive
drive.mount('/content/drive')

# Please adapt the root path based on your working environment
florence = "Colab Notebooks/Vegetation loss/mining/"
root = "/content/drive/MyDrive/" + florence

Mounted at /content/drive


In [ ]:
import geopandas as gpd
import pandas as pd

# Load the Jasanski geodataframe
gdf_path = root + 'open_database_mine_production/data/facilities.gpkg'
gdf = gpd.read_file(gdf_path)

# Load Jasanski ownership dataset
own_path = root + 'open_database_mine_production/data/ownership.csv'
own = pd.read_csv(own_path)

# Load the Maus geodataframe
gdf2_path = root + 'Maus-etal_2022_V2_allfiles/global_mining_polygons_v2.gpkg'
gdf2 = gpd.read_file(gdf2_path)


In [ ]:
# Check that crs of both gdf are the same
print(gdf.crs)
print(gdf2.crs)

EPSG:4326
EPSG:4326


## Data cleaning

### Create the parent and subsite ids

In [ ]:
# Create the parent and subsite ids
gdf[['parent_facility_id', 'subsite_facility_id']] = gdf['facility_id'].str.split('.', expand=True)
own[['parent_facility_id', 'subsite_facility_id']] = own['facility_id'].str.split('.', expand=True)
gdf[gdf["parent_facility_id"] == "COM00005"]

,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,facility_type,primary_commodity,commodities_products,facility_equipment,production_start,...,GID_0,GID_1,GID_2,GID_3,GID_4,source_id,comment,geometry,parent_facility_id,subsite_facility_id
4,COM00005.00,Achinsk Alumina Refinery,None,None,None,Refinery,Processing,"Alumina, Calcined soda",Sinter plant,1970.0,...,RUS,RUS.35_1,RUS.35.2_1,RUS.35.2.1_1,None,det_1289,None,MULTIPOINT (90.42397 56.23464),COM00005,00


### Clean the ownership table

#### Modify strings for pattern identification

In [ ]:
# Adjustments for fitting the patterns below
# Define replacement patterns
replacements = [
    ('%) and', '%),'),
    ('Medu Capital (15%) Minerales Y Productos Derivados S.A. (30%)', 'Medu Capital (15%), Minerales Y Productos Derivados S.A. (30%)'),
    ('Yancoal Australia Ltd (51%), Glencore Coal Pty Ltd. (49%).', 'Yancoal Australia Ltd (51%), Glencore Coal Pty Ltd. (49%)'),
    ('Sibanye-Stillwater (50%), Impala Platinum (50%(', 'Sibanye-Stillwater (50%), Impala Platinum (50%)'),
    ('Yancoal Australia Ltd (84.47%), Nippon Steel and Sumitomo Metal Australia Pty Limited (9.53%), Mitsubishi Materials (Australia) Pty Limited (6%).',
     'Yancoal Australia Ltd (84.47%), Nippon Steel and Sumitomo Metal Australia Pty Limited (9.53%), Mitsubishi Materials (Australia) Pty Limited (6%)'),
    ('Acacia Mining plc, 63.9% subsidiary of Barrick Gold Corporation', 'Acacia Mining plc (63.9%), subsidiary of Barrick Gold Corporation'),
    ('Bulyanhulu Gold Mine Limited (BGML) subsidiary of Acacia Mining plc (63.9%), subsidiary of Barrick Gold Corporation',
     'Bulyanhulu Gold Mine Limited (BGML) (63.9%), subsidiary of Acacia Mining plc subsidiary of Barrick Gold Corporation'),
    ('Pangea Minerals Limited (PML) subsidiary of Acacia Mining plc (63.9%), subsidiary of Barrick Gold Corporation',
     'Pangea Minerals Limited (PML) (63.9%), subsidiary of Acacia Mining plc subsidiary of Barrick Gold Corporation'),
    ('PT Nusa Halmahera Minerals (Newcrest Mining Limited 75%)', 'PT Nusa Halmahera Minerals (Newcrest Mining Limited) (75%)'),
    ('Freeport-McMoRan Inc. (72%), Sumitomo Metal Mining Arizona, Inc. (15%), SMM Morenci, Inc. (13%)',
     'Freeport-McMoRan Inc. (72%), Sumitomo Metal Mining Arizona Inc. (15%), SMM Morenci Inc. (13%)'),
    ('Lundin Mining Corp. (80%), Sumitomo Metal MiningCo., Ltd and Sumitomo Corporation (20%)',
     'Lundin Mining Corp. (80%), Sumitomo Metal MiningCo. Ltd and Sumitomo Corporation (20%)'),
    ('BHP (57.5%), Rio Tinto (30%), JECO Corporation consortium comprising Mitsubishi, JX Nippon Mining and Metals (10%), JECO2 Ltd (2.5%)',
     'BHP (57.5%), Rio Tinto (30%), JECO Corporation consortium comprising Mitsubishi JX Nippon Mining and Metals (10%), JECO2 Ltd (2.5%)'),
    ('Compañia de MinasBuenaventura S.A.A.', 'Compania de Minas Buenaventura S.A.A.'),
    ('Gecamines SA', 'Gecamines S.A.'),
    ("Newmont Mining Corporation", "Newmont Mining Corp.")
]

# Apply replacements using a loop
for old, new in replacements:
    own['owners'] = own['owners'].str.replace(old, new, regex=False)

#### Keep only the most recent owners

In [ ]:
# Keep only most recent owners
# Step 1: Sort the DataFrame by 'facility_id' and 'year' to make sure years are ordered correctly
df_sorted = own.sort_values(by=['facility_id', 'year'])

# Step 2: Group by 'facility_id' and keep only the row with the most recent year in the main DataFrame
# This will move all older years to the `previous_owners` DataFrame
# `idxmax()` keeps the index of the maximum (most recent) year in each group
most_recent_indices = df_sorted.groupby('facility_id')['year'].idxmax()

# Create the main DataFrame with only the most recent owners
own_recent = df_sorted.loc[most_recent_indices].reset_index(drop=True)

# Step 3: Identify older entries for the `previous_owners` DataFrame
previous_owners = df_sorted.drop(most_recent_indices).reset_index(drop=True)

#### Allocate main-site owners when subsite owners is missing

In [ ]:
# Loop through each row in gdf to find facility_ids that do not have an owner
for _, row in gdf.iterrows():
    requested_facility_id = row['facility_id']
    requested_parent_id = row['parent_facility_id']
    requested_subsite_id = row['subsite_facility_id']

    # Check if the requested facility_id is present in own_recent
    if requested_facility_id not in own_recent['facility_id'].values:
        # If not, check if there is a row in own_recent where the parent_facility_id matches and subsite_facility_id is '00'
        match = own_recent[(own_recent['parent_facility_id'] == requested_parent_id) &
                            (own_recent['subsite_facility_id'] == '00')]

        # If such a match exists, add a new row in own_recent for this subsite
        if not match.empty:
            # Copy the matching row
            parent_row = match.iloc[0].copy()

            # Modify the `facility_id` and `subsite_facility_id` to the current subsite
            parent_row['facility_id'] = requested_facility_id
            parent_row['subsite_facility_id'] = requested_subsite_id

            # Append the new row to own_recent
            own_recent = pd.concat([own_recent, pd.DataFrame([parent_row])], ignore_index=True)

#### Allocate main-site production start when subsite production start is missing

In [ ]:
# Step 1: Identify rows without a 'production_start' (NaN values)
missing_production_start = gdf[gdf['production_start'].isna()]

# Step 2: Loop through rows without 'production_start' and find matching parent rows with '00' subsite
for idx, row in missing_production_start.iterrows():
    parent_id = row['parent_facility_id']

    # Find the parent facility with subsite_facility_id == '00' for this parent_id
    parent_row = gdf[(gdf['parent_facility_id'] == parent_id) & (gdf['subsite_facility_id'] == '00')]

    # If the parent has a 'production_start' value, copy it to the current row
    if not parent_row['production_start'].isna().values[0]:
        # Update the original DataFrame with the parent's 'production_start'
        gdf.at[idx, 'production_start'] = parent_row['production_start'].values[0]

#### Allocate main-site geometry when subsites production start are missing

In [ ]:
# Step 1: Identify rows with empty geometry
missing_geometry = gdf[gdf['geometry'].is_empty]

# Step 2: Loop through rows with empty geometry and find matching parent rows with subsite_facility_id == '00' that have a valid geometry
for idx, row in missing_geometry.iterrows():
    parent_id = row['parent_facility_id']

    # Find the parent facility with subsite_facility_id == '00' for this parent_id
    parent_rows = gdf[(gdf['parent_facility_id'] == parent_id) & (gdf['subsite_facility_id'] == '00')]

    # Check if there are any rows for this parent_id that have a non-empty geometry
    if not gdf[(gdf['parent_facility_id'] == parent_id) & ((gdf['subsite_facility_id'] != "00") & (gdf['subsite_facility_id'] != row["subsite_facility_id"])) & (~gdf['geometry'].is_empty)].empty:
        # Skip this parent group if there's at least one row with non-empty geometry
        continue

    # If we reach this point, it means the only matching rows for this parent have empty geometry
    # Replace the geometry of the current row with the geometry of the parent row
    if not parent_rows.empty:
        # Use the first matching parent's geometry for replacement
        gdf.at[idx, 'geometry'] = parent_rows.iloc[0]['geometry']

#### Restructure the ownership table based on string patterns

In [ ]:
# Clean data and strings
pd.options.mode.chained_assignment = None

# Pattern 1: Multiple owners with associated ownership in parenthesis and separated by a comma

# Step 1: Identify rows where the format matches "owner1 (ownership1%), owner2 (ownership2%)"
pattern1 = r'^([\w\s\.,&-’\'()]+\(\d+\.?\d*%\))(, [\w\s\.,&-’\'()]+\(\d+\.?\d*%\))+$'
rows_pattern1 = own_recent[own_recent['owners'].str.match(pattern1, na=False)]

# Step 2: Split the 'owners' column for the identified rows
# This split will separate based on commas followed by space, where each segment is "Owner (Ownership%)"
split_owners = rows_pattern1['owners'].str.split(r',\s*(?![^()]*\))', expand=True).stack().reset_index(level=1, drop=True)

# Step 3: Create a new DataFrame to hold the split 'owners' column
own_pattern1 = rows_pattern1.drop(columns=['owners']).join(split_owners.rename('owners')).reset_index(drop=True)

# Step 4: Extract ownership percentages using regex and assign them to a new 'ownership' column
own_pattern1['ownership'] = own_pattern1['owners'].str.extract(r'(\d+\.?\d*)%')[0]

# Step 5: Clean up the 'owners' column by removing ownership percentages, leaving only owner names
own_pattern1['owners'] = own_pattern1['owners'].str.replace(r'\s*\(\d+\.?\d*%\)\s*', '', regex=True).str.strip()


# Pattern 2: Single owner with associated ownership in parenthesis

# Step 1: Identify rows where the format matches "owner1 (ownership1%)"
# Use a regex pattern to filter these rows
pattern2 = r'^[\w\s\.,&-:’\'()]+\(\d+\.?\d*%\)$'
own_pattern2 = own_recent[(~own_recent['owners'].str.match(pattern1, na=False)) & (own_recent['owners'].str.match(pattern2, na=False))]

# Step 2: Extract ownership percentages using regex and assign them to a new 'ownership' column
own_pattern2['ownership'] = own_pattern2['owners'].str.extract(r'\((\d+\.?\d*)%\)')[0]

# Step 3: Remove ownership percentage values from the 'owners' column, leaving only the owner names
own_pattern2['owners'] = own_pattern2['owners'].str.replace(r'\s*\(\d+\.?\d*%\)\s*', '', regex=True).str.strip()


# Pattern 3: Single owner with comment at the end of the string

# Step 1: Identify rows where the format matches "owner1 (ownership1%), comment"
# Use a regex pattern to filter these rows
pattern3 = r'^\w[\w\s\.,&-]+\(\d+\.?\d*%\), .+|^\w[\w\s\.,&-]+\([\w\s]+\)\s*\(\d+\.?\d*%\), .+'
own_pattern3 = own_recent[(~own_recent['owners'].str.match(pattern1, na=False)) & (~own_recent['owners'].str.match(pattern2, na=False)) & (own_recent['owners'].str.match(pattern3, na=False))]

# Step 2: Extract ownership percentages using regex and assign them to a new 'ownership' column
own_pattern3['ownership'] = own_pattern3['owners'].str.extract(r'\((\d+\.?\d*)%\)')[0]

# Step 3: Extract comments from the owners column using regex
# Comments start after a comma and space following the ownership percentage
own_pattern3['comment'] = own_pattern3['owners'].str.extract(r', (.+)$')[0]

# Step 4: Remove ownership percentage and comment values from the 'owners' column, leaving only the owner names
own_pattern3['owners'] = own_pattern3['owners'].str.replace(r'\s*\(\d+\.?\d*%\), .+', '', regex=True).str.strip()

own_treated = pd.concat([own_pattern1, own_pattern2, own_pattern3], ignore_index=True)

# Ajustments post treatment
own_treated['owners'] = own_treated['owners'].str.replace('After 2007: Goldcorp', 'Goldcorp', regex=False)

# Note: the remaining two sites missed information regarding the owner or the ownership.

### Some descriptive stats

In [ ]:
# Extract the unique facility_id that have one or several owners
final_comp = pd.read_csv(root + "final_comp.csv")

sites_with_owners = own_treated[own_treated["owners"].isin(final_comp["owners"].to_list())][["facility_id", "owners"]].drop_duplicates()
number_sites_agg = sites_with_owners.groupby("owners").agg({"facility_id": "count"})

print("Average number of sites per company", number_sites_agg["facility_id"].mean())
print("Total number of companies", sites_with_owners["owners"].drop_duplicates().count())
print("Total number of unique sites", sites_with_owners["facility_id"].drop_duplicates().count())
number_sites_agg

Average number of sites per company 9.897435897435898
Total number of companies 39
Total number of unique sites 366


,facility_id
owners,
African Rainbow Minerals Ltd.,7
Agnico Eagle Mines Limited,15
Antofagasta Plc.,5
Austral Gold Ltd.,4
BHP,53
Barrick Gold Corporation,17
CITIC Limited,1
Cleveland-Cliffs Inc.,13
Coronado Global Resources Inc.,7


### Filter site based on data availability

#### Filter 1: Data availability for production start date

The original gdf has 2413 observations. After this allocating the facility production start to subsite that do not have any and filtering on this basis, 845 observations are remaining.

Without the allocation, 598 observations are remaining.

In [ ]:
gdf_filtered = gdf[gdf["production_start"].notna()]
gdf_filtered

,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,facility_type,primary_commodity,commodities_products,facility_equipment,production_start,...,GID_0,GID_1,GID_2,GID_3,GID_4,source_id,comment,geometry,parent_facility_id,subsite_facility_id
2,COM00003.00,Absaloka,None,None,None,Mine,Coal,Sub-bituminous coal,None,1974.0,...,USA,USA.27_1,USA.27.2_1,None,None,det_1427,None,MULTIPOINT (-107.0829 45.80531),COM00003,00
4,COM00005.00,Achinsk Alumina Refinery,None,None,None,Refinery,Processing,"Alumina, Calcined soda",Sinter plant,1970.0,...,RUS,RUS.35_1,RUS.35.2_1,RUS.35.2.1_1,None,det_1289,None,MULTIPOINT (90.42397 56.23464),COM00005,00
7,COM00008.00,Aguablanca,None,None,None,Mine,Nickel,"Nickel, Copper, Cobalt, Gold, PGM",Underground,2005.0,...,ESP,ESP.11_1,ESP.11.1_1,ESP.11.1.10_1,ESP.11.1.10.11_1,det_1205,None,MULTIPOINT (-6.18033 37.96558),COM00008,00
8,COM00009.00,Ahafo,None,None,None,Mine,Gold,Gold,"Open pit, Underground",2006.0,...,GHA,GHA.2_1,GHA.2.3_1,None,None,det_1001,None,"MULTIPOINT (-2.36132 6.99983, -2.30703 7.08545)",COM00009,00
14,COM00015.00,Aktogay,None,None,None,Mine,Copper,"Copper, Silver, Copper cathode, Gold","Open pit, Heap leaching plant",2015.0,...,KAZ,KAZ.5_1,KAZ.5.2_1,None,None,det_1118,None,MULTIPOINT (79.93182 46.95243),COM00015,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,COM01407.05,Yuzhkuzbassugol,None,Yerunakovskaya-VIII,Yerunakovskaya-8,Mine,Coal,Metallurgical coal,Underground,2013.0,...,RUS,None,None,None,None,det_1556,None,POINT (87.3595 54.12133),COM01407,05
2403,COM01418.01,Zeravshan,None,Jilau,None,Mine,Gold,None,None,2007.0,...,TJK,None,None,None,None,det_1313,None,POINT (67.71722 39.35471),COM01418,01
2404,COM01418.02,Zeravshan,None,Taror,None,Mine,Gold,None,None,2007.0,...,TJK,None,None,None,None,det_1313,None,POINT (67.72896 39.35077),COM01418,02
2405,COM01418.03,Zeravshan,None,Khirskhona,None,Mine,Gold,None,None,2007.0,...,TJK,None,None,None,None,det_1313,None,POINT (67.74695 39.3902),COM01418,03


#### Filter 2: Data availability on ownership

After the filtering the sample goes from 845 to 514

Without the allocation of production start, the sample goes from 598 to 341 observations.

In [ ]:
# Extract the unique facility_id that have one or several owners
facility_ids_own = own_treated['facility_id'].unique()

gdf_filtered2 = gdf_filtered[gdf_filtered['facility_id'].isin(facility_ids_own)]
# Note: the difference between this approach and mapping with parent_facility_id is that 5 facilities 00 are not considered in the sample. They do
# not have an owner but their subsites do. The subsites are already included in the database and therefore do not need to be considered.  Plus 4 of them are
# filtered out in the polygon matching.

#### Filter 3: Data availability with polygons

After this filtering the sample goes from 514 to 257 observations.


Note: The default behavior of geopandas.sjoin (with predicate="intersects") does not automatically check whether all points in a MultiPoint geometry are within a polygon. It checks only based on the entire MultiPoint geometry, rather than assessing individual points within it. This means that if any one of the points in a MultiPoint geometry intersects a polygon, the join will consider it a match. With "within" as a predicate, we ensure that we select only polygons that contains all points within a Multipoint geometry.

If we opt for the predicate "intersects", the sample sites is of 298 observations.


In [ ]:
# Get the id of the facilities whose geometry is completely within a polygon of gdf2
matched_gdf = gpd.sjoin(gdf_filtered2, gdf2, how='inner', predicate='within')
ids = matched_gdf["facility_id"].unique() # 257 facilities
len(ids)

In [ ]:
# WITHOUT OWNERSHIP FILTER
# matched_gdf = gpd.sjoin(gdf_filtered, gdf2, how='inner', predicate='within')
# ids = matched_gdf["facility_id"].unique() # 257 facilities
# len(ids)

441

In [ ]:
# Add the polygons of the facilities whose geometry is completely within a polygon of gdf2 to the main dataframe
matched_gdf2 = gpd.sjoin(gdf2, gdf_filtered[gdf_filtered["facility_id"].isin(ids)], how='inner', predicate='intersects')

matched_gdf2[["facility_id", "parent_facility_id", "subsite_facility_id"]]

,facility_id,parent_facility_id,subsite_facility_id
10,COM01350.01,COM01350,01
10,COM01350.00,COM01350,00
81,COM00758.01,COM00758,01
81,COM00758.00,COM00758,00
243,COM00117.00,COM00117,00
...,...,...,...
44520,COM00020.00,COM00020,00
44523,COM01064.01,COM01064,01
44523,COM01064.02,COM01064,02
44523,COM00938.00,COM00938,00


#### Key only sub sites when they are available

In [ ]:
# Step 1: Identify parent_facility_ids that have more than one unique subsite_facility_id
# Create a Series counting the number of unique subsite_facility_id per parent_facility_id
subsite_count_per_parent = matched_gdf2.groupby('parent_facility_id')['subsite_facility_id'].nunique()

# Step 2: Identify parent_facility_ids with only one subsite_facility_id (i.e., only '00')
multiple_subsite_parents = subsite_count_per_parent[subsite_count_per_parent > 1].index

# Step 3: Drop rows with subsite_facility_id == '00' if they are in the list of single_subsite_parents
matched_gdf3 = matched_gdf2[~((matched_gdf2['subsite_facility_id'] == '00') & (matched_gdf2['parent_facility_id'].isin(multiple_subsite_parents)))]

# Reset index after filtering
matched_gdf3 = matched_gdf3.reset_index(drop=True)
matched_gdf3[["facility_id", "parent_facility_id", "subsite_facility_id"]]



,facility_id,parent_facility_id,subsite_facility_id
0,COM01350.01,COM01350,01
1,COM00758.01,COM00758,01
2,COM00117.00,COM00117,00
3,COM00875.00,COM00875,00
4,COM00768.00,COM00768,00
...,...,...,...
384,COM00020.00,COM00020,00
385,COM01064.01,COM01064,01
386,COM01064.02,COM01064,02
387,COM00938.00,COM00938,00


### Check duplicates and adjust sample

#### Check 1: There are no polygon duplicates.

False. We therefore drop sites for the following reasons:

- **Reason 1**: Subsites (0X) or main-sites (00) within one facility have the same geometry and same ownership structure: we keep only one subsite - the lowest number i.e. 01 if 01 and 02 exist or 00 for 00 and 04) (Total of 12 sites dropped)

- **Reason 2**: Subsites (0X) or main-sites (00) within different facilities have the same geometry and same ownership structure: we keep only one facility - the one with the lowest value in facility_id) (Total of 7 sites dropped)

- **Reason 3**: Subsites (0X) or main-sites (00) within different facilities have the same geometry but different ownership structures. We drop all sites.(Total of 5 sites dropped)

We now have one polygon per site.


In [ ]:
# Show duplicated polygons
duplicates = matched_gdf3[matched_gdf3.duplicated(subset="geometry", keep=False)][["facility_id", "parent_facility_id", "subsite_facility_id", "geometry", "facility_type", "facility_equipment", "country"]].sort_values(by="geometry")
print(len(duplicates[["facility_id", "geometry"]]))

46


In [ ]:
# Manual assignment of duplicates based on associated ownership structure in own_treated

# Reason 1: Subsites (0X) or main-sites (00) within one facility have the same geometry and same ownership structure: we keep only one subsite - the lowest number i.e. 01 if 01 and 02 exist or 00 for 00 and 04)
to_drop1 = ["COM00272.02", "COM00962.02", "COM00583.02", "COM00205.06", "COM01019.03", "COM01016.03", "COM01016.04", "COM00850.02"]
# Reason 2: Subsites (0X) or main-sites (00) within different facilities have the same geometry and same ownership structure: keep one facility - the one with the lowest value in facility_id)
to_drop2 = ["COM00824.00", "COM01282.00", "COM01134.00", "COM00969.00", "COM00489.01", "COM00489.02"]
# Reason 3: Subsites (0X) or main-sites (00) within different facilities have the same geometry but different ownership structures: drop them altogether
to_drop3 = ["COM00774.01", "COM00774.02", "COM01114.00", "COM00624.02", "COM00662.00"]
to_drop = to_drop1 + to_drop2 + to_drop3

matched_gdf4 = matched_gdf3[~matched_gdf3["facility_id"].isin(to_drop)] # WITHOUT OWNERSHIP FILTER ADD THIS: .drop_duplicates(subset='geometry', keep='first')

duplicates = matched_gdf4[matched_gdf4.duplicated(subset="geometry", keep=False)][["facility_id", "parent_facility_id", "subsite_facility_id", "geometry", "facility_type", "facility_equipment", "country"]].sort_values(by="geometry")


389
361
19


### Drop sites which started before the start of Landsat 7

In [ ]:
matched_gdf5 = matched_gdf4[matched_gdf3["production_start"] >= 2001]
print(len(matched_gdf5))

192


/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


#### Check 2: The polygons do not overlap.

True, there is only two polygons that overlap in the Maus et al. database and these geometries are not used in the working database.

In [ ]:
# Find overlaps within the GeoDataFrame by doing a self-overlay
overlapping_areas = gpd.overlay(gdf2, gdf2, how='intersection')

# Identify the geometry where there is an overlap in the Maus et al database
overlapping_geometry = overlapping_areas[overlapping_areas["AREA_1"] != overlapping_areas["AREA_2"]].geometry
overlapping_geometry

# Check if the geometry is found in the current working database
matched_gdf5[matched_gdf5["geometry"].isin(overlapping_geometry)]

<ipython-input-60-b7609c2ad520>:2: UserWarning: `keep_geom_type=True` in overlay resulted in 102 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlapping_areas = gpd.overlay(gdf2, gdf2, how='intersection')


,ISO3_CODE,COUNTRY_NAME,AREA,geometry,index_right,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,...,country,GID_0,GID_1,GID_2,GID_3,GID_4,source_id,comment,parent_facility_id,subsite_facility_id


In [ ]:
gdf_treated = matched_gdf5
own_treated = own_treated[own_treated["facility_id"].isin(gdf_treated["facility_id"])].drop(columns="id")
gdf_treated

In [ ]:
# Test on the owners

own_treated_ = pd.concat([own_pattern1, own_pattern2, own_pattern3], ignore_index=True)

# Ajustments post treatment
own_treated_['owners'] = own_treated_['owners'].str.replace('After 2007: Goldcorp', 'Goldcorp', regex=False)

own_treated_test = own_treated_[own_treated_["facility_id"].isin(matched_gdf4["facility_id"])].drop(columns="id")
own_treated_test[["owners"]].drop_duplicates()

,owners
12,BHP
13,Mitsubishi Development
14,Idemitsu Australia Resources
15,Chugoku Electric Power Australia Resources Pty...
16,NS Boggabri Pty Limited
...,...
1028,PT Indika Energy Tbk.
1122,OZ Minerals Ltd.
1183,Aeris Resources Ltd.
1187,PT Borneo Lumbung Energi & Metal Tbk.


### Extract

In [ ]:
# gdf_treated.to_csv(root + "gdf_treated.csv", index=False)
# gdf_treated.to_file(root + "gdf_treated.gpkg", driver="GPKG")
# own_treated.to_csv(root + "own_treated.csv", index=False)

## Analyse data

#### gdf_treated

We have 116 sites (incl. main-sites (00) and subsites(0X)) and 108 facilities (i.e. unique COM00XXXX).

In [ ]:
gdf_treated[gdf_treated["facility_id"] == "COM01052.01"][["facility_id", "production_start"]]

,facility_id,production_start
138,COM01052.01,2013.0


#### own_treated

In [ ]:
len(own_treated["owners"].unique())

72

In [ ]:
own_treated[["owners"]].drop_duplicates().sort_values(by="owners").to_csv(root + "owners.csv")